In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
jm1 = pd.read_csv(r'/content/jm1_csv.csv')
jm1=jm1.replace('?',0)
print('jm1 shape: ', jm1.shape)
jm1.head()

jm1 shape:  (10885, 22)


,loc,v(g),ev(g),iv(g),n,v,l,d,i,e,...,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,defects
0,1.1,1.4,1.4,1.4,1.3,1.30,1.30,1.30,1.30,1.30,...,2,2,2,2,1.2,1.2,1.2,1.2,1.4,False
1,1.0,1.0,1.0,1.0,1.0,1.00,1.00,1.00,1.00,1.00,...,1,1,1,1,1,1,1,1,1,True
2,72.0,7.0,1.0,6.0,198.0,1134.13,0.05,20.31,55.85,23029.10,...,51,10,8,1,17,36,112,86,13,True
3,190.0,3.0,1.0,3.0,600.0,4348.76,0.06,17.06,254.87,74202.67,...,129,29,28,2,17,135,329,271,5,True
4,37.0,4.0,1.0,4.0,126.0,599.12,0.06,17.19,34.86,10297.30,...,28,1,6,0,11,16,76,50,7,True


In [3]:
jm1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10885 entries, 0 to 10884
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   loc                10885 non-null  float64
 1   v(g)               10885 non-null  float64
 2   ev(g)              10885 non-null  float64
 3   iv(g)              10885 non-null  float64
 4   n                  10885 non-null  float64
 5   v                  10885 non-null  float64
 6   l                  10885 non-null  float64
 7   d                  10885 non-null  float64
 8   i                  10885 non-null  float64
 9   e                  10885 non-null  float64
 10  b                  10885 non-null  float64
 11  t                  10885 non-null  float64
 12  lOCode             10885 non-null  int64  
 13  lOComment          10885 non-null  int64  
 14  lOBlank            10885 non-null  int64  
 15  locCodeAndComment  10885 non-null  int64  
 16  uniq_Op            108

In [4]:
target = 'defects'
y = jm1[target]
x=jm1.drop(['defects'], axis=1)
x.shape

(10885, 21)

In [5]:
y.shape

(10885,)

In [6]:
from sklearn.feature_selection import VarianceThreshold

var_thres=VarianceThreshold(threshold=1) # threshold=1 means it will tell features with 1 variance. We can change threshold value as per need
var_thres.fit(x)



VarianceThreshold(threshold=1)

In [7]:
var_thres.get_support() # We will remove false features 

array([ True,  True,  True,  True,  True,  True, False,  True,  True,
        True, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

In [8]:
remove_columns = [column for column in x.columns
                    if column not in x.columns[var_thres.get_support()]]

print(len(remove_columns))

print("name of columns to be removed: -")

for column in remove_columns:
    print(column)

2
name of columns to be removed: -
l
b


In [9]:
x.shape

(10885, 21)

In [10]:
x=x.drop(remove_columns,axis=1)
# droping columns 

In [11]:
x.shape

(10885, 19)

In [12]:
y.shape

(10885,)

In [13]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, shuffle=True)

In [14]:
print(x.shape)
print(y.shape)
print('-------------------')
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(10885, 19)
(10885,)
-------------------
(8708, 19)
(8708,)
(2177, 19)
(2177,)


In [15]:
from sklearn.svm import SVC
from sklearn import metrics #for checking the model accuracy
svc = SVC()
svc.fit(x_train, y_train)
svc_pred=svc.predict(x_test)
from sklearn import metrics #for checking the model accuracy
acc_svm = metrics.accuracy_score(svc_pred,y_test)
print('The accuracy of the SVM is:',acc_svm)
print('Th confusion matrix of SVM:\n ',metrics.confusion_matrix(y_test, svc_pred))

The accuracy of the SVM is: 0.8066146072576941
Th confusion matrix of SVM:
  [[1749    9]
 [ 412    7]]


In [16]:
print(metrics.confusion_matrix(y_test, svc_pred))
print(metrics.classification_report(y_test, svc_pred))

[[1749    9]
 [ 412    7]]
              precision    recall  f1-score   support

       False       0.81      0.99      0.89      1758
        True       0.44      0.02      0.03       419

    accuracy                           0.81      2177
   macro avg       0.62      0.51      0.46      2177
weighted avg       0.74      0.81      0.73      2177



In [17]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=2) #put 2 neighbors because we have 2 labels.
knn.fit(x_train,y_train)
knn_pred = knn.predict(x_test)
acc_knn = metrics.accuracy_score(knn_pred,y_test)
print('The accuracy of the KNN is', acc_knn)
print('The confusion matrix of KNN:\n ',metrics.confusion_matrix(y_test, knn_pred))
print(metrics.confusion_matrix(y_test, knn_pred))
print(metrics.classification_report(y_test, knn_pred))

The accuracy of the KNN is 0.8038585209003215
The confusion matrix of KNN:
  [[1692   66]
 [ 361   58]]
[[1692   66]
 [ 361   58]]
              precision    recall  f1-score   support

       False       0.82      0.96      0.89      1758
        True       0.47      0.14      0.21       419

    accuracy                           0.80      2177
   macro avg       0.65      0.55      0.55      2177
weighted avg       0.76      0.80      0.76      2177



In [18]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
model.fit(x_train, y_train)
y_predict = model.predict(x_test)
model_score = model.score(x_test, y_test)
print(model_score)
print(metrics.confusion_matrix(y_test, y_predict))
print(metrics.classification_report(y_test, y_predict))

0.7588424437299035
[[1494  264]
 [ 261  158]]
              precision    recall  f1-score   support

       False       0.85      0.85      0.85      1758
        True       0.37      0.38      0.38       419

    accuracy                           0.76      2177
   macro avg       0.61      0.61      0.61      2177
weighted avg       0.76      0.76      0.76      2177



In [19]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(x_train, y_train)
model.score(x_train, y_train) 

0.8039733578318787

In [20]:
test_pred = model.predict(x_test)
print(metrics.classification_report(y_test, test_pred))
print(metrics.confusion_matrix(y_test, test_pred))

              precision    recall  f1-score   support

       False       0.83      0.96      0.89      1758
        True       0.49      0.15      0.22       419

    accuracy                           0.81      2177
   macro avg       0.66      0.55      0.56      2177
weighted avg       0.76      0.81      0.76      2177

[[1694   64]
 [ 358   61]]
